<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Домашнее-Задание-#2" data-toc-modified-id="Домашнее-Задание-#2-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Домашнее Задание #2</a></span></li></ul></div>

# Домашнее Задание #2
+ Собрать информацию о всех строящихся объектах на сайте "наш.дом.рф"
+ Cохранить ее в pandas dataframe
+ Cохранить pandas dataframe в excel
+ Cохранить pandas dataframe в pickle
+ Cохранить pandas dataframe в БД


In [1]:
import requests,math
from tqdm.auto import tqdm

In [2]:
offset_ = 1 # размер шага
limit_ = 100 # количество объектов в выдаче
url = f'https://xn--80az8a.xn--d1aqf.xn--p1ai/%D1%81%D0%B5%D1%80%D0%B2%D0%B8%D1%81%D1%8B/api/kn/object?offset={offset_}&limit={limit_}&sortField=devId.devShortCleanNm&sortType=asc&objStatus=0'

In [3]:
# Отправляем GET запрос в api.hh.ru
res = requests.get(url) 
# Преобразовываем ответ api из JSON в словарь dict()
objects_data = res.json()
#количество объектов всего
objects_data['data']['total']

11656

In [4]:
#считаем количество страниц
num_pages=math.ceil(objects_data['data']['total']/limit_)
num_pages

117

In [5]:
# Запускаем цикл для сбора всех идентификаторов объектов со всех страниц
all_object_ids= []
for offset_ in tqdm(range(num_pages)):
    url = f'https://xn--80az8a.xn--d1aqf.xn--p1ai/%D1%81%D0%B5%D1%80%D0%B2%D0%B8%D1%81%D1%8B/api/kn/object?offset={offset_}&limit={limit_}&sortField=devId.devShortCleanNm&sortType=asc&objStatus=0'
    res = requests.get(url)
    objects_data = res.json() 
    objids = [x['objId'] for x in objects_data.get('data').get('list')]
    # заполняем список элементами objects с помощью метода extend
    all_object_ids.extend(objids)

  0%|          | 0/117 [00:00<?, ?it/s]

In [6]:
len(all_object_ids)

11700

In [7]:
# Собираем массив с полными описаниями строящихся объектов по идентификаторам
full_object = []
for obj_id in tqdm(all_object_ids):
    url = f'https://xn--80az8a.xn--d1aqf.xn--p1ai/%D1%81%D0%B5%D1%80%D0%B2%D0%B8%D1%81%D1%8B/api/object/{obj_id}'
    res = requests.get(url)
    objects = res.json()
    full_object.append(objects)

  0%|          | 0/11700 [00:00<?, ?it/s]

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [99]:
full_object

[{'data': {'id': 13399,
   'hobjId': 34754,
   'pdId': 28141,
   'developer': {'devId': 306,
    'devShortCleanNm': '2МЕН ГРУПП',
    'devShortNm': '2МЕН ГРУПП',
    'devFullCleanNm': '2МЕН ГРУПП',
    'problObjCnt': 0,
    'buildObjCnt': 5,
    'comissObjCnt': 0,
    'regRegionDesc': 'Тюменская область',
    'devPhoneNum': '+7(345)279-18-88',
    'devSite': '2mengroup.ru',
    'devEmail': 'info@2mengroup.ru',
    'devInn': '7701651356',
    'devOgrn': '1067746424899',
    'devKpp': '720301001',
    'devLegalAddr': 'Тюменская область, город Тюмень, улица Комсомольская дом 75/5',
    'devFactAddr': 'Тюменская область, город Тюмень, улица Комсомольская 75/5',
    'lastRpdId': 802817,
    'fundGuarantyFlg': 1,
    'devOrgRegRegionCd': 72,
    'devEmplMainFullNm': 'Киселев Сергей Михайлович',
    'developerGroupName': '2МЕН ГРУПП ДЕВЕЛОПМЕНТ',
    'orgForm': {'id': 1,
     'fullForm': 'Акционерное общество',
     'shortForm': 'АО'},
    'companyGroupId': 5776,
    'objGuarantyEscrowFlg': 0

In [18]:
# Создаем таблицу DataFrame из всех полученных данных

In [100]:
import pandas as pd

In [101]:
df = pd.json_normalize(full_object)

In [102]:
df.head()

,errcode,data.id,data.hobjId,data.pdId,data.developer.devId,data.developer.devShortCleanNm,data.developer.devShortNm,data.developer.devFullCleanNm,data.developer.problObjCnt,data.developer.buildObjCnt,...,data.metro.colors,data.transportDistIndex,data.transportDistIndexValue,data.complexShortNm,data.airQualityIndexValue,data.greenAreaIndexValue,data.developer.bankruptStage.bankruptStageCd,data.developer.bankruptStage.bankruptStageDesc,data.developer.bankruptStage.bankruptLawUrl,data.developer.orgBankruptMsgDttm
0,0,13399,34754,28141,306,2МЕН ГРУПП,2МЕН ГРУПП,2МЕН ГРУПП,0,5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0,13391,37219,28101,306,2МЕН ГРУПП,2МЕН ГРУПП,2МЕН ГРУПП,0,5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,0,13397,37201,28141,306,2МЕН ГРУПП,2МЕН ГРУПП,2МЕН ГРУПП,0,5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,0,13398,37165,28141,306,2МЕН ГРУПП,2МЕН ГРУПП,2МЕН ГРУПП,0,5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,0,13383,39024,28105,306,2МЕН ГРУПП,2МЕН ГРУПП,2МЕН ГРУПП,0,5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [96]:
df.shape

(216, 111)

In [98]:
# Сохраняем в csv файл
df.to_csv('object.csv', index=False)
# Сохраняем в эксель-файл
df.to_excel('object.xlsx', index=False)
# Сохраняем в файл в формате pickle
df.to_pickle('object.pkl')